# Roadway Network Search
The purpose of this notebook is to understand and visualize how Network Wrangler conducts searches. 
It can also be used as a template to troubleshoot or check specific searches.

In [7]:
import os
import warnings

import folium
import osmnx as ox
import pandas as pd

from network_wrangler import load_roadway_from_dir

%config IPCompleter.greedy=True
pd.set_option("display.max_columns", None)

warnings.filterwarnings("ignore")

# 1 - Load Network

 - Read the network files into the `RoadwayNetwork` class

In [8]:
STPAUL_DIR = os.path.join(os.path.dirname(os.getcwd()), "examples", "stpaul")
net = load_roadway_from_dir(STPAUL_DIR)

Skipping field outboundReferenceIds: unsupported OGR type: 5
Skipping field inboundReferenceIds: unsupported OGR type: 5


# 2 - Define Selections
  - can be represented by dictionaries
  - `link.name` is required unless a `link` unique ID is used (set at `links_df.params._addtl_explicit_ids`)
  - A and B nodes should be set with a unique identifier
  - If you want to qualify your selection farther based on attributes, you can add that as well

In [9]:
easy_selection = {
    "links": {
            "name": ["6th", "Sixth", "sixth"]
    },  # find streets that have one of the various forms of 6th
    "from": {"osm_node_id": "187899923"},  # start searching for segments at this ID
    "to": {"osm_node_id": "187865924"},  # end at this ID
}

longer_selection = {
    "links": {
            "name": ["6th", "Sixth", "sixth"]
    },  # find streets that have one of the various forms of 6th,
    "from": {"osm_node_id": "187899923"},  # start searching for segments at this ID
    "to": {"osm_node_id": "187942339"},  # end at this ID
}

multi_criteria_selection = {
    "links": {
        "name": ["6th", "Sixth", "sixth"],
        "lanes": [2, 3],  # from the initial selection, only select streets with 1 OR 2 lanes
    },
    "from": {"osm_node_id": "187899923"},  # start searching for segments at this id
    "to": {"osm_node_id": "187942339"},  # end segment at this id
}



# 3 - Run Selections


In [10]:
# Selecting all, including non roadway nodes. This is useful for creating a default value.
all_selection = {"links": {"all": True}}
selected_ix = net.get_selection(all_selection).selected_links
len(selected_ix) == len(net.links_df.of_type.drive_access)

True

In [11]:
multi_sel = net.get_selection(multi_criteria_selection)
multi_sel.selected_links_df[["name","lanes"]].head

<bound method NDFrame.head of                                     name  lanes
model_link_id_idx                              
78740                    East 5th Street      3
96983                    East 5th Street      3
134542                   East 5th Street      2
165718                   East 5th Street      3
336883             North Broadway Street      2
395710               Robert Street North      2>

# 4 - Visualizing the Selection

Selection visualization has been thrown into the class defined below `selection_map`.  In the future, this should be generalized and moved into Network Wrangler.

## Display Selection

Showing final selected links in green and candidate links in blue.

In [19]:
from_node = multi_sel.segment.segment_from_node_s
to_node = multi_sel.segment.segment_to_node_s
selected_links = multi_sel.selected_links_df
segment_links = multi_sel.segment.segment_links_df
candidate_links = multi_sel.segment.subnet.subnet_links_df


sel_map = candidate_links.explore(
    tiles = "CartoDB positron",
    color = "grey",
    marker_kwds={"line_opacity": 0.5, "weight": 10}
)
segment_links.explore(
    m = sel_map,
    color = "cyan",
    marker_kwds={"line_opacity": 0.5, "weight": 7}
)
selected_links.explore(
    m = sel_map,
    color = "blue",
    marker_kwds={"weight": 5}
)
import folium
folium.Marker(
    [from_node.geometry.y, from_node.geometry.x],
    icon=folium.Icon(icon="play"),
).add_to(sel_map)
folium.Marker(
    [to_node.geometry.y, to_node.geometry.x],
    icon=folium.Icon(icon="stop"),
).add_to(sel_map)
sel_map
